# Simple Machine Translation Project 2025
#### Tech Stack: Python, Hugging Face MarianMT, NLTK/ChatterBot/GPT-based chatbot

# Installed packages for this project

Packages Installed: transformers, torch, nltk, chatterbot, chatterbot_corpus, spacy

In [1]:
from transformers import MarianMTModel, MarianTokenizer

In [ ]:
# The MarianMT model is loaded for translating from source to the target language
def load_translation_model(src_lang="fr", tgt_lang="en"):
    model_name = f"Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}"
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return model, tokenizer
    
# The text is translated using the MarianMT model
def translate(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(**inputs)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text

In [ ]:
# Model
model_en_fr, tokenizer_en_fr = load_translation_model("en", "fr")  # English → French
model_fr_en, tokenizer_fr_en = load_translation_model("fr", "en")  # French → English

text = "The clowns had taken over. And yes, they were literally clowns."  \
"Over 100 had appeared out of a small VW bug that had been driven up to the bank." \
"Now they were all inside and had taken it over."
translated_text = translate(text, model_en_fr, tokenizer_en_fr)
print(f"Translated: {translated_text}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Translated: Les clowns s'étaient emparés.Et oui, ils étaient littéralement des clowns.Plus de 100 étaient apparus d'un petit bug VW qui avait été conduit jusqu'à la banque.Maintenant ils étaient tous à l'intérieur et l'avaient pris.


In [4]:
# Building the AI chatbot by utilizing the Chatterbot library
import spacy
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer


In [ ]:

nlp = spacy.load("en_core_web_sm")
print("SpaCy model loaded successfully!")

SpaCy model loaded successfully!


In [ ]:
chatbot = ChatBot('MultilingualBot')

trainer = ChatterBotCorpusTrainer(chatbot)
  
# Train the bot with multiple corpus
trainer.train("chatterbot.corpus.english.greetings",
              "chatterbot.corpus.english",
              "chatterbot.corpus.french",
              "chatterbot.corpus.english.conversations" )
  
def get_chatbot_response(user_input):
    return chatbot.get_response(user_input).text

 
user_input = "Quels sont tes centres d'interets"
response = get_chatbot_response(user_input)
print(f"Chatbot: {response}")


ChatterBot Corpus Trainer: 26it [00:03,  7.64it/s]

Chatbot: Je suis interesse par beaucoup de choses. Nous pouvons parler de n'importe quoi!


In [ ]:
# Connect chatbot with translation

def multilingual_chatbot(user_input, user_lang="fr"):
    model_user_to_en, tokenizer_user_to_en = load_translation_model(user_lang, "en")
    translated_input = translate(user_input, model_user_to_en, tokenizer_user_to_en)
    
    # Get chatbot response in English
    chatbot_response = get_chatbot_response(translated_input)
    
    # Translate chatbot response back to user language
    model_en_to_user, tokenizer_en_to_user = load_translation_model("en", user_lang)
    final_response = translate(chatbot_response, model_en_to_user, tokenizer_en_to_user)
    
    return final_response

user_message = "Bonjour! Comment ça va?"  # French input
response = multilingual_chatbot(user_message, user_lang="fr")
print(f"Chatbot: {response}")  

Chatbot: Je vais bien.
